In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import os
import skimage.filters

from google.colab import drive, files
drive.mount('/content/drive') # mount your google colab fil

Mounted at /content/drive


In [ ]:
images = []
labels = []
SIZE = 256

for directory_path in glob.glob("/content/drive/MyDrive/ML_IPData7/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
       # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        images.append(img.flatten())
        labels.append(label)

Rifle
Golf cart


In [ ]:
images = np.array(images)
type(labels)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(labels)
enc_labels = labelencoder.transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

thresh = VarianceThreshold(threshold=0.05)
images_thresh = thresh.fit_transform(images)

scalar = StandardScaler()

scaled_images = scalar.fit_transform(images_thresh)

pca = PCA(n_components=50)
images = pca.fit_transform(scaled_images)
X_train, X_test, y_train, y_test = train_test_split(images, enc_labels, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, utils
from tensorflow.keras.models import Sequential

y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(50,)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                     │ (None, 256)                 │          13,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 54,338 (212.26 KB)

 Trainable params: 54,338 (212.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)
mc = ModelCheckpoint('best_model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[es, mc])
model.evaluate(X_test, y_test)
print(history.history)

Epoch 1/20
25/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6721 - loss: 4.9181
Epoch 1: val_accuracy improved from -inf to 0.82993, saving model to best_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6830 - loss: 4.6706 - val_accuracy: 0.8299 - val_loss: 1.3632
Epoch 2/20
21/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8223 - loss: 1.8525  
Epoch 2: val_accuracy improved from 0.82993 to 0.92517, saving model to best_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8328 - loss: 1.7273 - val_accuracy: 0.9252 - val_loss: 0.4057
Epoch 3/20
23/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8795 - loss: 1.1381  
Epoch 3: val_accuracy improved from 0.92517 to 0.93197, saving model to best_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8748 - loss: 1.1390 - val_accuracy: 0.9320 - val_loss: 0.2994
Epoch 4/20
19/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8906 - loss: 0.9250 
Epoch 4: val_accuracy improved from 0.93197 to 0.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Accuracy: 0.9183673469387755
Recall: 0.810077519379845
F1 Score: 0.810077519379845
